In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from scipy.stats import norm, kstest, anderson
from scipy.optimize import minimize

# np.random.seed(42)
# plt.style.use("seaborn")

In [ ]:

ticker = "AAPL"
data = yf.download(ticker, start="2022-01-01", end="2023-01-01")["Adj Close"]

# Compute log returns
returns = np.log(data / data.shift(1)).dropna()

plt.figure(figsize=(10,4))
plt.plot(data, label="Stock Price")
plt.title(f"{ticker} Price")
plt.legend()
plt.show()

plt.figure(figsize=(10,4))
plt.hist(returns, bins=50, density=True, alpha=0.6, color='blue')
plt.title(f"Log Returns Distribution of {ticker}")
plt.show()


In [ ]:
mu_hat = returns.mean() * 252
sigma_hat = returns.std() * np.sqrt(252)

# Jump detection (3-sigma filter)
threshold = 3 * returns.std()
jumps = returns[np.abs(returns - returns.mean()) > threshold]

# Jump parameters
lam_hat = len(jumps) / len(returns) * 252
mu_j_hat = jumps.mean()
sigma_j_hat = jumps.std()

print("GBM Parameters:")
print(f"mu = {mu_hat:.4f}, sigma = {sigma_hat:.4f}")
print("\nMerton Jump Parameters:")
print(f"lambda = {lam_hat:.4f}, mu_j = {mu_j_hat:.4f}, sigma_j = {sigma_j_hat:.4f}")

In [ ]:
def simulate_gbm(S0, mu, sigma, T=1, steps=252, n_sims=10000):
    dt = T/steps
    prices = np.zeros((n_sims, steps+1))
    prices[:,0] = S0
    for t in range(1, steps+1):
        Z = np.random.normal(0,1,n_sims)
        prices[:,t] = prices[:,t-1] * np.exp((mu-0.5*sigma**2)*dt + sigma*np.sqrt(dt)*Z)
    return prices

def simulate_merton(S0, mu, sigma, lam, mu_j, sigma_j, T=1, steps=252, n_sims=10000):
    dt = T/steps
    prices = np.zeros((n_sims, steps+1))
    prices[:,0] = S0
    for t in range(1, steps+1):
        Z = np.random.normal(0,1,n_sims)
        diffusion = (mu-0.5*sigma**2)*dt + sigma*np.sqrt(dt)*Z
        N_t = np.random.poisson(lam*dt, n_sims)
        J_t = np.random.normal(mu_j, sigma_j, n_sims) * N_t
        prices[:,t] = prices[:,t-1] * np.exp(diffusion + J_t)
    return prices

In [ ]:
S0 = data.iloc[-1]
T = 1
steps = 252
n_sims = 10000

gbm_prices = simulate_gbm(S0, mu_hat, sigma_hat, T, steps, n_sims)
merton_prices = simulate_merton(S0, mu_hat, sigma_hat, lam_hat, mu_j_hat, sigma_j_hat, T, steps, n_sims)

plt.figure(figsize=(12,6))
for i in range(20):
    plt.plot(gbm_prices[i], alpha=0.6)
plt.title("GBM Sample Paths")
plt.show()

plt.figure(figsize=(12,6))
for i in range(20):
    plt.plot(merton_prices[i], alpha=0.6)
plt.title("Merton Jump-Diffusion Sample Paths")
plt.show()

In [ ]:
gbm_final = gbm_prices[:,-1]
merton_final = merton_prices[:,-1]

# Convert to returns
gbm_returns = np.log(gbm_final/S0)
merton_returns = np.log(merton_final/S0)

# KS Test
ks_gbm = kstest(returns, 'norm', args=(gbm_returns.mean(), gbm_returns.std()))
ks_merton = kstest(returns, 'norm', args=(merton_returns.mean(), merton_returns.std()))

print("Kolmogorov–Smirnov Test Results:")
print(f"GBM KS-stat = {ks_gbm.statistic:.4f}, p-value = {ks_gbm.pvalue:.4f}")
print(f"Merton KS-stat = {ks_merton.statistic:.4f}, p-value = {ks_merton.pvalue:.4f}")